# Creating a STAC using PySTAC

Notebook going through the basic of creating a SpatioTemporal Asset Catalog using Python 3. Full details at are in the documentation for PySTAC 

## Install and load 

In [22]:
pip install pystac[validation] # install pystac

You should consider upgrading via the '/Users/oskarfkrauss/opt/anaconda3/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pystac

## Obtain data and metadata 

In [4]:
import csv
import gzip
from io import StringIO
from urllib.request import urlopen

# Collection 1 data
url = 'https://landsat-pds.s3.amazonaws.com/c1/L8/scene_list.gz'

# Read and unzip the content
response = urlopen(url)
gunzip_response = gzip.GzipFile(fileobj=response)
content = gunzip_response.read()

# Read the scenes in as dictionaries
scenes = list(csv.DictReader(StringIO(content.decode("utf-8"))))

In [5]:
len(scenes) # lots of them!

2355938

In [6]:
scenes[0] # look at what scenes are

{'productId': 'LC08_L1TP_149039_20170411_20170415_01_T1',
 'entityId': 'LC81490392017101LGN00',
 'acquisitionDate': '2017-04-11 05:36:29.349932',
 'cloudCover': '0.0',
 'processingLevel': 'L1TP',
 'path': '149',
 'row': '39',
 'min_lat': '29.22165',
 'min_lon': '72.41205',
 'max_lat': '31.34742',
 'max_lon': '74.84666',
 'download_url': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/149/039/LC08_L1TP_149039_20170411_20170415_01_T1/index.html'}

Filter to ones which contain a certain location

In [8]:
lat, lon = 55.9422, -3.1823
location_name = "Edinburgh Flat"
filter_year = '2020'

In [9]:
def keep_scene(scene): # function that does the filtering
    contains_location = float(scene['min_lat']) < lat and float(scene['max_lat']) > lat and \
                        float(scene['min_lon']) < lon and float(scene['max_lon']) > lon
    is_correct_year = '{}-'.format(filter_year) in scene['acquisitionDate']
    return contains_location and is_correct_year
        
location_scenes = [scene for scene in scenes if keep_scene(scene)]

In [10]:
len(location_scenes) # fewer of them, woop

58

In [13]:
scene = location_scenes[0]
scene

{'productId': 'LC08_L1GT_205021_20200106_20200106_01_RT',
 'entityId': 'LC82050212020006LGN00',
 'acquisitionDate': '2020-01-06 11:16:07.471447',
 'cloudCover': '94.11',
 'processingLevel': 'L1GT',
 'path': '205',
 'row': '21',
 'min_lat': '54.78572',
 'min_lon': '-5.63922',
 'max_lat': '56.97768',
 'max_lon': '-1.73361',
 'download_url': 'https://s3-us-west-2.amazonaws.com/landsat-pds/c1/L8/205/021/LC08_L1GT_205021_20200106_20200106_01_RT/index.html'}

Now extract metadata (additional data about scenes) from MTL file. This file is alongside the `download_url` and so we need a funciton to extract necessary pieces (from scenes) and then retrive MTL. 

In [11]:
def get_asset_url(scene, suffix):
    product_id = scene['productId']
    download_url = scene['download_url']    
    asset_filename = '{}_{}'.format(product_id, suffix)
    return download_url.replace('index.html', asset_filename)

Now read url

In [18]:
mtl_url = get_asset_url(scene, 'MTL.txt')
print(pystac.STAC_IO.read_text(mtl_url))

GROUP = L1_METADATA_FILE
  GROUP = METADATA_FILE_INFO
    ORIGIN = "Image courtesy of the U.S. Geological Survey"
    REQUEST_ID = "0502001063447_00003"
    LANDSAT_SCENE_ID = "LC82050212020006LGN00"
    LANDSAT_PRODUCT_ID = "LC08_L1GT_205021_20200106_20200106_01_RT"
    COLLECTION_NUMBER = 01
    FILE_DATE = 2020-01-06T15:25:28Z
    STATION_ID = "LGN"
    PROCESSING_SOFTWARE_VERSION = "LPGS_13.1.0"
  END_GROUP = METADATA_FILE_INFO
  GROUP = PRODUCT_METADATA
    DATA_TYPE = "L1GT"
    COLLECTION_CATEGORY = "RT"
    ELEVATION_SOURCE = "GLS2000"
    OUTPUT_FORMAT = "GEOTIFF"
    SPACECRAFT_ID = "LANDSAT_8"
    SENSOR_ID = "OLI_TIRS"
    WRS_PATH = 205
    WRS_ROW = 21
    NADIR_OFFNADIR = "NADIR"
    TARGET_WRS_PATH = 205
    TARGET_WRS_ROW = 21
    DATE_ACQUIRED = 2020-01-06
    SCENE_CENTER_TIME = "11:16:07.4714479Z"
    CORNER_UL_LAT_PRODUCT = 56.97768
    CORNER_UL_LON_PRODUCT = -5.63922
    CORNER_UR_LAT_PRODUCT = 56.99834
    CORNER_UR_LON_PRODUCT = -1.66005
    CORNER_LL_LAT_PRODU

In [20]:
def get_metadata(url):
    """ Convert Landsat MTL file to dictionary of metadata values """
    mtl = {}
    mtl_text = pystac.STAC_IO.read_text(url)
    for line in mtl_text.split('\n'):
        meta = line.replace('\"', "").strip().split('=')
        if len(meta) > 1:
            key = meta[0].strip()
            item = meta[1].strip()
            if key != "GROUP" and key != "END_GROUP":
                mtl[key] = item
    return mtl


In [21]:
metadata = get_metadata(mtl_url)
metadata

{'ORIGIN': 'Image courtesy of the U.S. Geological Survey',
 'REQUEST_ID': '0502001063447_00003',
 'LANDSAT_SCENE_ID': 'LC82050212020006LGN00',
 'LANDSAT_PRODUCT_ID': 'LC08_L1GT_205021_20200106_20200106_01_RT',
 'COLLECTION_NUMBER': '01',
 'FILE_DATE': '2020-01-06T15:25:28Z',
 'STATION_ID': 'LGN',
 'PROCESSING_SOFTWARE_VERSION': 'LPGS_13.1.0',
 'DATA_TYPE': 'L1GT',
 'COLLECTION_CATEGORY': 'RT',
 'ELEVATION_SOURCE': 'GLS2000',
 'OUTPUT_FORMAT': 'GEOTIFF',
 'SPACECRAFT_ID': 'LANDSAT_8',
 'SENSOR_ID': 'OLI_TIRS',
 'WRS_PATH': '205',
 'WRS_ROW': '21',
 'NADIR_OFFNADIR': 'NADIR',
 'TARGET_WRS_PATH': '205',
 'TARGET_WRS_ROW': '21',
 'DATE_ACQUIRED': '2020-01-06',
 'SCENE_CENTER_TIME': '11:16:07.4714479Z',
 'CORNER_UL_LAT_PRODUCT': '56.97768',
 'CORNER_UL_LON_PRODUCT': '-5.63922',
 'CORNER_UR_LAT_PRODUCT': '56.99834',
 'CORNER_UR_LON_PRODUCT': '-1.66005',
 'CORNER_LL_LAT_PRODUCT': '54.78572',
 'CORNER_LL_LON_PRODUCT': '-5.49447',
 'CORNER_LR_LAT_PRODUCT': '54.80475',
 'CORNER_LR_LON_PRODUCT': 

Now have everything me need to make STAC item...

## Make STAC Item 

Items have several required fields including `id`, `datetime`, and `bbox` (bounding box) so we first need to extract those from the metadata. This can be done using the following functions.  

In [26]:
# id

def get_id(metadata): 
    return metadata['LANDSAT_SCENE_ID'][:-5]

# datetime

from dateutil.parser import parse

def get_datetime(metadata):
    return parse('%sT%s' % (metadata['DATE_ACQUIRED'], metadata['SCENE_CENTER_TIME']))

# bbox

def get_bbox(metadata):
    coords = [[
        [float(metadata['CORNER_UL_LON_PRODUCT']), float(metadata['CORNER_UL_LAT_PRODUCT'])],
        [float(metadata['CORNER_UR_LON_PRODUCT']), float(metadata['CORNER_UR_LAT_PRODUCT'])],
        [float(metadata['CORNER_LR_LON_PRODUCT']), float(metadata['CORNER_LR_LAT_PRODUCT'])],
        [float(metadata['CORNER_LL_LON_PRODUCT']), float(metadata['CORNER_LL_LAT_PRODUCT'])],
        [float(metadata['CORNER_UL_LON_PRODUCT']), float(metadata['CORNER_UL_LAT_PRODUCT'])]
    ]]
    lats = [c[1] for c in coords[0]]
    lons = [c[0] for c in coords[0]]
    return [min(lons), min(lats), max(lons), max(lats)]

# geometry, this is a bit trickier since landsat tiles are tilted

def get_geometry(scene, bbox):
    url = get_asset_url(scene, 'ANG.txt')
    sz = []
    coords = []
    ang_text = pystac.STAC_IO.read_text(url)
    for line in ang_text.split('\n'):
        if 'BAND01_NUM_L1T_LINES' in line or 'BAND01_NUM_L1T_SAMPS' in line:
            sz.append(float(line.split('=')[1]))
        if 'BAND01_L1T_IMAGE_CORNER_LINES' in line or 'BAND01_L1T_IMAGE_CORNER_SAMPS' in line:
            coords.append([float(l) for l in line.split('=')[1].strip().strip('()').split(',')])
        if len(coords) == 2:
            break
    dlon = bbox[2] - bbox[0]
    dlat = bbox[3] - bbox[1]
    lons = [c/sz[1] * dlon + bbox[0] for c in coords[1]]
    lats = [((sz[0] - c)/sz[0]) * dlat + bbox[1] for c in coords[0]]
    coordinates = [[
        [lons[0], lats[0]], [lons[1], lats[1]], [lons[2], lats[2]], [lons[3], lats[3]], [lons[0], lats[0]]
    ]]
    
    return {'type': 'Polygon', 'coordinates': coordinates}

Then store variables...

In [41]:
item_id = get_id(metadata)
print('ID:', item_id)

item_datetime = get_datetime(metadata)
print('datetime:', item_datetime)

item_bbox = get_bbox(metadata)
print('bbox:', item_bbox)

item_geometry = get_geometry(scene, item_bbox) # from scene as opposed to metadata
print('geometry:', item_geometry)

ID: LC82050212020006
datetime: 2020-01-06 11:16:07.471447+00:00
bbox: [-5.63922, 54.78572, -1.66005, 56.99834]
geometry: {'type': 'Polygon', 'coordinates': [[[-4.679899357070097, 56.99739750867762], [-1.6618520817312392, 56.48293191696963], [-2.6246412167403794, 54.78628975250376], [-5.637228781570707, 55.30986728082811], [-4.679899357070097, 56.99739750867762]]]}


Can now double check we have the correct area by formatting the item geometry as GeoJSON then copying into a GeoJSON reader (such as http://geojson.io).

In [42]:
import json

print(json.dumps(item_geometry, indent=2))

{
  "type": "Polygon",
  "coordinates": [
    [
      [
        -4.679899357070097,
        56.99739750867762
      ],
      [
        -1.6618520817312392,
        56.48293191696963
      ],
      [
        -2.6246412167403794,
        54.78628975250376
      ],
      [
        -5.637228781570707,
        55.30986728082811
      ],
      [
        -4.679899357070097,
        56.99739750867762
      ]
    ]
  ]
}


No we have everything we need to make a STAC item! We create it below using pystac.

In [49]:
item = pystac.Item(id = item_id,
                  datetime = item_datetime,
                  bbox = item_bbox,
                  geometry = item_geometry,
                  properties = {})

In [51]:
item.validate() #PySTAC function to check we're doing things correctly

No errors, all good!

So far we only have the required attributes, however it is often a good idea to add other common features, such as ground sample distance. We also enable the `eo` extenstion (not 100% on this) so that we can include the cloud cover data.

In [54]:
item.common_metadata.gsd = 30.0 # not all bands are 30.0 but we can adjust this when we add the asset later

In [58]:
item.ext.enable('eo')

def get_cloud_cover(metadata):
    return float(metadata['CLOUD_COVER'])

item.ext.eo.cloud_cover = get_cloud_cover(metadata)
item.ext.eo.cloud_cover

94.11

Now that we have set up the item, we can add assets, i.e. the actual files.

### Adding assets